In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import os
import pandas as pd
from utils.misc import get_summary, get_query

os.chdir('/home/s2310409/workspace/coliee-2024/')



In [2]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def load_data(dir):
    with open(dir, 'r') as fp:
        train_data = json.load(fp)

    data = []
    for key in train_data.keys():
        data.append([key, train_data[key]])

    return pd.DataFrame(data, columns=['source', 'target'])

data_df = load_data(f'dataset/test.json')
data_df

,source,target
0,070318.txt,[015076.txt]
1,077960.txt,"[009054.txt, 040860.txt]"
2,042319.txt,"[093691.txt, 075956.txt, 084953.txt, 022987.txt]"
3,041766.txt,[039269.txt]
4,077407.txt,[038669.txt]
...,...,...
314,085079.txt,"[044669.txt, 003144.txt]"
315,031370.txt,"[096341.txt, 060602.txt, 047107.txt, 084522.tx..."
316,085828.txt,"[004301.txt, 074887.txt, 088994.txt]"
317,024957.txt,"[015009.txt, 080348.txt]"


In [11]:
with open('dataset/c2023/bm25_candidates_test.json', 'r') as fp:
    candidate_dict = json.load(fp)

In [18]:

# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

def reranking_prompting(list_articles, query_content):
    prompting = f"In bellow articles:  "
    for a_id in list_articles:
        a_content = get_summary(a_id)
        prompting = prompting + f"\n##Article {a_id}: {a_content},"
        
    prompting  = prompting +f"\n##Question: which articles really relevant to the following article? \n##Article: {query_content}"
    return prompting

source = data_df['source'][0]
text = reranking_prompting(candidate_dict[source][1:3], get_summary(source))

encodeds = tokenizer(text, return_tensors="pt")
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [19]:
print(decoded[0].replace(text, ''))

<s> 

1. What are the proper steps to be taken by a Board member or Hearing Officer upon receiving information which may lead to a reasonable apprehension of bias, such as a communication between a Board member and a staff member or a communication between a staff member and a witness?
2. Is it reasonable to require that a party bring an application for judicial review within 30 days of decision?
3. When can the provisions of Guideline 7 not be applied?

Relevant Articles:

* 071237.txt: The applicant argues that the Board's interventions were intrusive and intimidating, interfering with his ability to properly present his case.
* 19716.txt: A party may request material relevant to an application that is in the possession of a tribunal whose order is the subject of the application.

Both articles discuss issues related to fairness in legal proceedings, with the first article focusing on the conduct of Board members during a hearing, and the second article discussing the request for mat

In [20]:
candidate_dict[source][1:3]

['071237.txt', '019716.txt']